## Registro de tablas Delta en el HIve metastore de Databricks CE

En el código siguiente, se registran las tablas Delta de votantes y llamadas en el Hive Metastore de Databricks CE, para habilitar su
consulta a través del lenguaje SQL

In [ ]:
from pyspark.sql import SparkSession

# 1) Crear la sesión Spark
spark = (SparkSession.builder
         .appName("Registro_Tablas_Delta")
         .enableHiveSupport()  # habilita Hive metastore
         .getOrCreate())

# 2) Crear la base de datos lógica para aislar el DW
spark.sql("""
    CREATE DATABASE IF NOT EXISTS call_center_db
    LOCATION '/mnt/Data Call Center/warehouse/call_center_db.db'
""")

# 3) Registrar la tabla Delta de VOTANTES
spark.sql("""
    CREATE TABLE IF NOT EXISTS call_center_db.silver_voters
    USING DELTA
    LOCATION '/mnt/Data Call Center/Silver/Voters/'
""")

# 4) Registrar la tabla Delta de LLAMADAS
spark.sql("""
    CREATE TABLE IF NOT EXISTS call_center_db.silver_calls
    USING DELTA
    LOCATION '/mnt/Data Call Center/Silver/Calls/'
""")

# 5) Una vez registradas las tablas Delta, almacenaremos una vista que combina las tablas. Esta acompañará las predicciones del modelo de Ml que se entrenará
# para mostrar por votante y franja horaria, el número telefónico óptimo que maximiza su probabilidad de contestación
spark.sql("""
    CREATE OR REPLACE VIEW call_center_db.v_response_by_voter_and_time AS
    SELECT
        v.vendor_lead_code,
        v.full_name,
        v.district_number,
        v.age,
        v.gender,
        c.call_date,
        HOUR(TO_TIMESTAMP(c.call_date, 'yyyy-MM-dd HH:mm:ss')) AS call_hour,
        c.phone_number_dialed,
        c.status,
        CASE WHEN LOWER(c.status) IN ('answered', 'completed') THEN 1 ELSE 0 END AS answered
    FROM call_center_db.silver_calls c
    LEFT JOIN call_center_db.silver_voters v
    ON c.vendor_lead_code = v.vendor_lead_code
""")

# 6) Consultas opcionales para verificar
spark.sql("SHOW TABLES IN call_center_db").show()
spark.sql("DESCRIBE DETAIL call_center_db.silver_voters").show(truncate=False)
spark.sql("DESCRIBE DETAIL call_center_db.silver_calls").show(truncate=False)